In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

In [ ]:
data_reviews = pd.read_excel('/content/Reviews_information_cleaned_data_final.xlsx')
data_reviews.head()

,Unnamed: 0,rating,customer name,review title,reviews,preposition_linkage,preposition_linkage_checked,Noun_Verb_Noun,Noun_Verb_Noun_checked,adjective_linkage,adjective_linkage_checked,likes,dislikes,date
0,0,3,Amit Patil,Does the job,the product is not very good . it is just ok. ...,[],[],['it feel well.installation'] --> well install...,['well installation'],"['ingenious service', well installation']","['ingenious service', 'well installation']",413,233,2018-05-25
1,1,3,md.shakeel ahmed,Nice,reviewing after using 15 days what i observed ...,"['amount of stuff', 'installations over purcha...","['amount of stuff', 'installations over purcha...","['sleep crack sound', 'we dismantle bed', 'u s...","['sleep crack sound', 'we dismantle bed', 'you...","['head support', 'installation may b', 'storag...","['head support', 'improper installation', 'sto...",152,100,2018-08-25
2,2,5,Samtabdi Nayak,Best in the market!,i don’t know all above or below comments but f...,"['product within range', 'range of', 'product ...",['product within range'],"['product put inches', 'i have insecurity', 'i...","['product put inches', 'i have insecurity', 'i...","['best product', 'such good purchase', 'little...","['best product', 'such good purchase', 'little...",20,8,2019-01-25
3,3,4,ranjeeta sabat,Really Nice,i m giving my review after 10 days. the bed lo...,"['akhilesh came.the', 'gap of floor', 'demand ...","['good in price', 'gap of floor', 'demand for ...","['i give review', 'ksjb160729 instal bed', 'he...",[],"['days.the beds', 'inch mattressmine', 'perfec...","['perfect storage space', 'white heels', 'more...",20,17,2020-06-25
4,4,5,Rahul Mohandas Nair,Must buy!,the product is satisfactory! i would say the b...,"['installation of bed', 'screws on joints', 'j...","['installation of bed', 'screws on joints', 'j...",['i get matters'],[],"['build quality', 'price range', 'extra screws...","['build quality', 'price range', 'extra screws...",20,7,2020-11-25


In [ ]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
import spacy
from tqdm import tqdm
#nlp = spacy.load('en_core_web_lg', parse=True, tag=True, entity=True)
nlp = spacy.load("en_core_web_sm")

     |████████████████████████████████| 12.0MB 301kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-cp37-none-any.whl size=12019125 sha256=85861f801874816364d50d593550fefbce667b44996708f255fe17473dd6c961
  Stored in directory: /root/.cache/pip/wheels/48/5c/1c/15f9d02afc8221a668d2172446dd8467b20cdb9aef80a172a4
Successfully built en-core-web-sm


In [ ]:
txt = 'Great car and has long range'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [ ]:
txt =  "wonderful to drive the camry "
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [ ]:
## Below code extracts the aspect data and also extracts related adjectives, adverbs or nouns. 
## The data also extracts clustered nouns. But since clustered nouns don't have much polarity values and most of the nouns are extracted in some way, so we drop the idea of using 
## clustered values to find polarity values. The nouns are already used for feature selection and all other extracted methods have nouns in some form or another

aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(data_reviews['reviews']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(data_reviews['reviews'][x])) != 0:
        lines = str(data_reviews['reviews'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
        for line in lines:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])
            
    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
data_reviews['compound_nouns'] = comp_terms
data_reviews['aspect_keywords'] = aspect_terms
data_reviews['competition'] = enemy
data_reviews['competition_comp_nouns'] = ecomp_terms
data_reviews['competition_aspects'] = easpect_terms
data_reviews.head()

100%|██████████| 2506/2506 [03:35<00:00, 11.64it/s]


,Unnamed: 0,rating,customer name,review title,reviews,preposition_linkage,preposition_linkage_checked,Noun_Verb_Noun,Noun_Verb_Noun_checked,adjective_linkage,adjective_linkage_checked,likes,dislikes,date,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects
0,0,3,Amit Patil,Does the job,the product is not very good . it is just ok. ...,[],[],['it feel well.installation'] --> well install...,['well installation'],"['ingenious service', well installation']","['ingenious service', 'well installation']",413,233,2018-05-25,[],"[very good purse, feeling well, good purse]",[],[],[]
1,1,3,md.shakeel ahmed,Nice,reviewing after using 15 days what i observed ...,"['amount of stuff', 'installations over purcha...","['amount of stuff', 'installations over purcha...","['sleep crack sound', 'we dismantle bed', 'u s...","['sleep crack sound', 'we dismantle bed', 'you...","['head support', 'installation may b', 'storag...","['head support', 'improper installation', 'sto...",152,100,2018-08-25,"[(head support, support), (nut bolts, bolts), ...","[proper installation, bad persons, even kneel,...",[],[],[]
2,2,5,Samtabdi Nayak,Best in the market!,i don’t know all above or below comments but f...,"['product within range', 'range of', 'product ...",['product within range'],"['product put inches', 'i have insecurity', 'i...","['product put inches', 'i have insecurity', 'i...","['best product', 'such good purchase', 'little...","['best product', 'such good purchase', 'little...",20,8,2019-01-25,"[(ok product, product), (engineering ply, ply)...","[good materials, sometime hurts, little insecu...",[],[],[]
3,3,4,ranjeeta sabat,Really Nice,i m giving my review after 10 days. the bed lo...,"['akhilesh came.the', 'gap of floor', 'demand ...","['good in price', 'gap of floor', 'demand for ...","['i give review', 'ksjb160729 instal bed', 'he...",[],"['days.the beds', 'inch mattressmine', 'perfec...","['perfect storage space', 'white heels', 'more...",20,17,2020-06-25,"[(inch mattress, mattress), (storage space, sp...","[easily came, pressure immediately, actually i...",[],[],[]
4,4,5,Rahul Mohandas Nair,Must buy!,the product is satisfactory! i would say the b...,"['installation of bed', 'screws on joints', 'j...","['installation of bed', 'screws on joints', 'j...",['i get matters'],[],"['build quality', 'price range', 'extra screws...","['build quality', 'price range', 'extra screws...",20,7,2020-11-25,"[(build quality, quality), (price range, range...","[extra screws, fits perfectly, tough time]",[],[],[]


In [ ]:
data_reviews.to_excel('aspect_data_extracted.xlsx')

In [ ]:
data_reviews.head(1000)

,Unnamed: 0,rating,customer name,review title,reviews,preposition_linkage,preposition_linkage_checked,Noun_Verb_Noun,Noun_Verb_Noun_checked,adjective_linkage,adjective_linkage_checked,likes,dislikes,date,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects
0,0,3,Amit Patil,Does the job,the product is not very good . it is just ok. ...,[],[],['it feel well.installation'] --> well install...,['well installation'],"['ingenious service', well installation']","['ingenious service', 'well installation']",413,233,2018-05-25,[],"[very good purse, feeling well, good purse]",[],[],[]
1,1,3,md.shakeel ahmed,Nice,reviewing after using 15 days what i observed ...,"['amount of stuff', 'installations over purcha...","['amount of stuff', 'installations over purcha...","['sleep crack sound', 'we dismantle bed', 'u s...","['sleep crack sound', 'we dismantle bed', 'you...","['head support', 'installation may b', 'storag...","['head support', 'improper installation', 'sto...",152,100,2018-08-25,"[(head support, support), (nut bolts, bolts), ...","[proper installation, bad persons, even kneel,...",[],[],[]
2,2,5,Samtabdi Nayak,Best in the market!,i don’t know all above or below comments but f...,"['product within range', 'range of', 'product ...",['product within range'],"['product put inches', 'i have insecurity', 'i...","['product put inches', 'i have insecurity', 'i...","['best product', 'such good purchase', 'little...","['best product', 'such good purchase', 'little...",20,8,2019-01-25,"[(ok product, product), (engineering ply, ply)...","[good materials, sometime hurts, little insecu...",[],[],[]
3,3,4,ranjeeta sabat,Really Nice,i m giving my review after 10 days. the bed lo...,"['akhilesh came.the', 'gap of floor', 'demand ...","['good in price', 'gap of floor', 'demand for ...","['i give review', 'ksjb160729 instal bed', 'he...",[],"['days.the beds', 'inch mattressmine', 'perfec...","['perfect storage space', 'white heels', 'more...",20,17,2020-06-25,"[(inch mattress, mattress), (storage space, sp...","[easily came, pressure immediately, actually i...",[],[],[]
4,4,5,Rahul Mohandas Nair,Must buy!,the product is satisfactory! i would say the b...,"['installation of bed', 'screws on joints', 'j...","['installation of bed', 'screws on joints', 'j...",['i get matters'],[],"['build quality', 'price range', 'extra screws...","['build quality', 'price range', 'extra screws...",20,7,2020-11-25,"[(build quality, quality), (price range, range...","[extra screws, fits perfectly, tough time]",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,4,Flipkart Customer,Very Good,edges are not cut perfectly .. safe for kiss ....,['perfectly.unsafe for quality.we'],['unsafe for kids'],['edges cut perfectly.unsafe'],['sharp edges'],['kids.not good quality.we'],['worst for kids'],0,0,2020-08-25,[],"[good quality, cut perfectly]",[],[],[]
996,996,5,Kishor Devadiga,Worth every penny,not so bad items..... installation is properly...,[],[],[],[],['bad items'],['bad items'],0,0,2020-08-25,[],"[properly done, bad items]",[],[],[]
997,997,3,Flipkart Customer,Does the job,good one,[],[],[],[],[],[],0,0,2020-08-25,[],[],[],[],[]
998,998,4,Flipkart Customer,Value-for-money,"vice product, but height is less.",[],[],[],[],['nice product'],['nice product'],0,0,2020-08-25,"[(vice product, product)]",[],[],[],[]
